In [4]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
import collections
import firebase_admin
from firebase_admin import credentials, firestore
import pickle
import faiss


# Initialize Firebase
service_account_key_path = 'C:/Users/hp/Downloads/archive (4)/login-162b0-firebase-adminsdk-ajpww-6d2c012515.json'
cred = credentials.Certificate(service_account_key_path)
# firebase_admin.initialize_app(cred)
db = firestore.client()

def collect_image_paths_and_labels(root_dir):
    image_paths = []
    labels = []
    for split in ['train', 'val']:
        split_dir = os.path.join(root_dir, split)
        for class_name in os.listdir(split_dir):
            class_dir = os.path.join(split_dir, class_name)
            if os.path.isdir(class_dir):
                for img_name in os.listdir(class_dir):
                    img_path = os.path.join(class_dir, img_name)
                    if img_path.lower().endswith(('.jpg', '.jpeg', '.png')):
                        image_paths.append(img_path)
                        labels.append(class_name)
    return image_paths, labels

def preprocess_image(image_path, target_size=(224, 224)):
    img = Image.open(image_path).convert('RGB')
    img = img.resize(target_size, Image.LANCZOS)
    img_array = np.array(img)
    return img_array / 255.0

def image_batch_generator(image_files, batch_size, preprocess_function):
    num_images = len(image_files)
    for offset in range(0, num_images, batch_size):
        batch_files = image_files[offset:offset + batch_size]
        batch_images = [preprocess_function(img_path) for img_path in batch_files]
        yield np.array(batch_images)

def load_model():
    return MobileNetV2(weights='imagenet', include_top=False, pooling='avg')

def extract_features(image_files, model, batch_size=32):
    features = []
    for batch_images in tqdm(image_batch_generator(image_files, batch_size, preprocess_image), total=len(image_files) // batch_size):
        batch_features = model.predict(batch_images)
        features.extend(batch_features)
    return np.array(features)

def apply_pca(features):
    pca = PCA(n_components=256)
    reduced_features = pca.fit_transform(features)
    return pca, reduced_features

def cluster_features(features):
    kmeans = MiniBatchKMeans(n_clusters=100, random_state=42)
    labels = kmeans.fit_predict(features)
    pickle.dump(kmeans, open('kmeans_model.pkl', 'wb'))  # Save the KMeans model
    return labels

def build_faiss_index(features):
    dimension = features.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(features.astype(np.float32))
    return index

def store_image_metadata(db, image_paths, cluster_assignments):
    batch = db.batch()
    ref = db.collection('image_metadata')
    for idx, path in enumerate(image_paths):
        doc_ref = ref.document()
        batch.set(doc_ref, {
            'image_path': path,
            'cluster_id': int(cluster_assignments[idx])
        })
        print(1)
        if (idx + 1) % 500 == 0:
            batch.commit()
            batch = db.batch()
    batch.commit()


    






    

def main():
    dataset_dir = 'C:/Users/hp/Downloads/archive (4)/imagenet-mini'
    image_files, _ = collect_image_paths_and_labels(dataset_dir)
    model = load_model()
    features = extract_features(image_files, model)
    pca, reduced_features = apply_pca(features)
    labels = cluster_features(reduced_features)
    index = build_faiss_index(reduced_features)
    pickle.dump(pca, open('pca_model_1.pkl', 'wb'))
    faiss.write_index(index, 'faiss_index_1.index')
    store_image_metadata(db, image_files, labels)
    

if __name__ == '__main__':
    main()


C:\Users\hp\AppData\Local\Temp\ipykernel_21552\1974959299.py:51: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV2(weights='imagenet', include_top=False, pooling='avg')
  0%|          | 0/1208 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step


  0%|          | 1/1208 [00:05<1:48:20,  5.39s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 942ms/step


  0%|          | 2/1208 [00:06<1:02:28,  3.11s/it]

...

100%|██████████| 1208/1208 [26:35<00:00,  1.47s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


1209it [26:38,  1.32s/it]                          


1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


DeadlineExceeded: 504 Deadline Exceeded

In [3]:
import streamlit as st
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
import pickle
import faiss
import firebase_admin
from firebase_admin import credentials, firestore

# Initialize Firebase
cred = credentials.Certificate('C:/Users/hp/Downloads/archive (4)/login-162b0-firebase-adminsdk-ajpww-6d2c012515.json')
firebase_admin.initialize_app(cred)
db = firestore.client()

# Load models and indices
model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')
pca = pickle.load(open('pca_model.pkl', 'rb'))  # Load PCA model
kmeans = pickle.load(open('kmeans_model.pkl', 'rb'))  # Load KMeans model

def preprocess_image(image_file):
    img = Image.open(image_file).convert('RGB')
    img = img.resize((224, 224))
    img_array = np.array(img)
    return img_array / 255.0

def extract_features(image_array):
    img_batch = np.expand_dims(image_array, axis=0)
    features = model.predict(img_batch)[0]
    reduced_features = pca.transform([features])
    return reduced_features[0]

def identify_cluster(reduced_features):
    cluster_id = kmeans.predict([reduced_features])[0]
    return cluster_id

def find_similar_images(cluster_id, query_features, num_results=5):
    faiss_index = faiss.read_index(f'faiss_indices/cluster_{cluster_id}.index')
    distances, indices = faiss_index.search(np.array([query_features], dtype='float32'), num_results)
    return indices[0]

def fetch_images_from_firebase(indices):
    image_paths = []
    for idx in indices:
        doc_ref = db.collection('image_metadata3').document(str(idx))
        doc = doc_ref.get()
        if doc.exists:
            image_paths.append(doc.to_dict()['image_path'])
    return image_paths

st.title('Cluster-based Image Similarity Search')
uploaded_file = st.file_uploader("Choose an image...", type=['jpg', 'jpeg', 'png'])
if uploaded_file is not None:
    image = preprocess_image(uploaded_file)
    query_features = extract_features(image)
    cluster_id = identify_cluster(query_features)
    indices = find_similar_images(cluster_id, query_features, num_results=5)
    image_paths = fetch_images_from_firebase(indices)

    st.write("Similar Images Found:")
    for path in image_paths:
        st.image(path)


ModuleNotFoundError: No module named 'streamlit'